In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 


In [4]:


# Charger le DataFrame 
df = pd.read_excel('Singles.xlsx')

# au préalable , j'ai ajouté quelques colonnes dans ma base de donnée à partir des autres bases fournies 
# Afficher les valeurs uniques des colonnes level et grade 
valeurs_uniques = df['Grade'].unique()
valeurs_uniques1 = df['Level'].unique()
print(valeurs_uniques)
print (valeurs_uniques1)


# S'assurer que la colonne 'Date' est au format datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Filtrer pour inclure uniquement les lignes entre le 1er janvier 2018 et le 31 décembre 2024
df = df[(df['Date'] >= '2018-01-01') & (df['Date'] <= '2024-12-31')]



[ 2.  3. nan  1.]
[ 5.  2.  3.  4. nan  1.  6.]


In [5]:

# Dictionnaire des multiplicateurs que j'ai créer seule 
multiplicateurs = {
    (0, 0): 0.5, (0, 1): 0.6, (0, 2): 0.7, (0, 3): 0.8, (0, 4): 0.9, (0, 5): 1.0, (0, 6): 1.1,
    (1, 0): 1.2, (1, 1): 1.3, (1, 2): 1.4, (1, 3): 1.5, (1, 4): 1.6, (1, 5): 1.7, (1, 6): 1.8,
    (2, 0): 1.9, (2, 1): 2.0, (2, 2): 2.2, (2, 3): 2.4, (2, 4): 2.6, (2, 5): 2.8, (2, 6): 3.0,
    (3, 0): 3.2, (3, 1): 3.5, (3, 2): 3.8, (3, 3): 4.0, (3, 4): 4.3, (3, 5): 4.6, (3, 6): 5.0
}


# Fonction pour obtenir le multiplicateur basé sur 'level' et 'grade'
def get_multiplicateur(row):
    return multiplicateurs.get((row['Grade'], row['Level']), 1.0)  # Valeur par défaut 1.0 si pas trouvé

# Appliquer la fonction pour créer la colonne 'multiplicateur'
df['multiplicateurs'] = df.apply(get_multiplicateur, axis=1)

# Afficher le DataFrame mis à jour
print(df['multiplicateurs'])

0        2.8
1        2.8
2        2.8
3        2.8
4        2.8
        ... 
78093    3.5
78094    3.5
78095    3.5
78096    1.0
78097    1.0
Name: multiplicateurs, Length: 78098, dtype: float64


In [6]:
# Création d'un DataFrame vide avec les colonnes 'grade', 'level' et 'multiplicateur'
data = []

# Ajouter toutes les combinaisons de grade et level avec leur multiplicateur pour une meilleure visualisation
for grade in range(4):
    for level in range(7):
        data.append([grade, level, multiplicateurs.get((grade, level))])

# Créer le DataFrame
dff = pd.DataFrame(data, columns=['Grade', 'Level', 'multiplicateurs'])

# Affichage du DataFrame
print(dff)

print(df.columns)



    Grade  Level  multiplicateurs
0       0      0              0.5
1       0      1              0.6
2       0      2              0.7
3       0      3              0.8
4       0      4              0.9
5       0      5              1.0
6       0      6              1.1
7       1      0              1.2
8       1      1              1.3
9       1      2              1.4
10      1      3              1.5
11      1      4              1.6
12      1      5              1.7
13      1      6              1.8
14      2      0              1.9
15      2      1              2.0
16      2      2              2.2
17      2      3              2.4
18      2      4              2.6
19      2      5              2.8
20      2      6              3.0
21      3      0              3.2
22      3      1              3.5
23      3      2              3.8
24      3      3              4.0
25      3      4              4.3
26      3      5              4.6
27      3      6              5.0
Index(['MatchI

In [7]:
# Afficher les valeurs distinctes d'une colonne (par exemple, 'Round')
valeurs_distinctes = df['Round'].unique()

# Affichage des valeurs distinctes
print(valeurs_distinctes)

['Qualification round of 16' 'Qualification quarter final' 'Round of 32'
 'Round of 16' 'Quarter final' 'Semi final' 'Final'
 'Qualification round of 64' 'Qualification round of 32' 'Round of 64'
 'Round of 128' '1' '2' '3' nan 'Qualification round of 128' 'Round 1'
 'Round 2' 'Round 3' '3rd/4th place' 'Round 4' 'Round 5' 'Round of 256']


In [9]:
# Dictionnaire de mappage pour encoder les valeurs de la colonne 'Round'
mapping = {
    'Qualification round of 16': 0,
    'Qualification quarter final': 0,
    'Qualification round of 64': 0,
    'Qualification round of 32': 0,
    'Qualification round of 128': 0,
    'Round of 256': 1,
    'Round of 128': 2,
    'Round of 64': 3,
    'Round of 32': 4,
    'Round of 16': 5,
    'Quarter final': 6,
    'Semi final': 7,
    'Final': 8,
    '1': 9,
    '2': 9,
    '3': 9,
    'Round 1': 9,
    'Round 2': 9,
    'Round 3': 9,
    'Round 4': 9,
    'Round 5': 9,
    '3rd/4th place': 10,
    None: -1,  # Pour les valeurs manquantes (nan)
    float('nan'): -1  # Encodage pour les NaN si pandas les traite explicitement
}

# Encodage de la colonne 'Round'
df['Encoded Round'] = df['Round'].map(mapping)

# Vérification des résultats
print(df[['Round', 'Encoded Round']].head(20))



# Encodage de la durée en catégories
def encode_duration(Duration):
    if Duration < 25:
        return 0
    elif 25 <= Duration <= 35:
        return 1
    elif 35 < Duration <= 45:
        return 2
    else:
        return 3

# Ajouter une colonne encodée pour Duration
df['Encoded Duration'] = df['Duration'].apply(encode_duration)

# Vérification des résultats
print(df[['Duration', 'Encoded Duration']])



# Étape 1 : Compter le nombre de victoires cumulées pour chaque joueur
victoires_par_joueur = df['WinnerId'].value_counts()

# Étape 2 : Associer les victoires cumulées à chaque joueur
df['Victoires Cumulées P1'] = df['Player1Id'].map(victoires_par_joueur).fillna(0).astype(int)
df['Victoires Cumulées P2'] = df['Player2Id'].map(victoires_par_joueur).fillna(0).astype(int)

# Étape 3 : Fonction pour classer les joueurs
def classer_joueur(victoires):
    if victoires <= 3:
        return 0  # Pas bon
    elif 4 <= victoires <= 6:
        return 1  # Moyennement bon
    elif 7 <= victoires <= 10:
        return 2  # Bon
    else:
        return 3  # Très bon

# Ajouter les colonnes de classement
df['Classement P1'] = df['Victoires Cumulées P1'].apply(classer_joueur)
df['Classement P2'] = df['Victoires Cumulées P2'].apply(classer_joueur)





df['Base Points'] = df['multiplicateurs'] * 10  # j'ai posé 10 pour un base que je multiplie par les multiplicateurs précedement calculé , en fonction du level et degré de l'épreuve 
df['PRG P1'] = df ['points p1'] / df ['Total '] # ces sdeux colonnes representent le ration des points , nombre de points marque par le joueur sur le total des points marqué par les deux joueurs 
df['PRG P2'] = df ['points p2'] / df ['Total ']




                          Round  Encoded Round
0     Qualification round of 16              0
1     Qualification round of 16              0
2   Qualification quarter final              0
3     Qualification round of 16              0
4     Qualification round of 16              0
5     Qualification round of 16              0
6   Qualification quarter final              0
7     Qualification round of 16              0
8     Qualification round of 16              0
9   Qualification quarter final              0
10    Qualification round of 16              0
11  Qualification quarter final              0
12                  Round of 32              4
13                  Round of 32              4
14                  Round of 32              4
15                  Round of 32              4
16                  Round of 32              4
17                  Round of 32              4
18                  Round of 32              4
19                  Round of 32              4
       Durati

0        0
1        0
2        0
3        0
4        0
        ..
78093    0
78094    0
78095    0
78096    4
78097    5
Name: Encoded Round, Length: 78098, dtype: int64


In [10]:
# Initialisation du score de base pour tous les joueurs
scores = {player: 1000 for player in set(df['Player1Id']).union(set(df['Player2Id']))}

# Fonction pour calculer le coefficient adversaire
def coef_adversaire(winner_class, loser_class):
    if winner_class > loser_class:
        return 1.1
    elif winner_class < loser_class:
        return 0.9
    else:
        return 1.0

# Calcul du nouveau score pour chaque match
def calculer_nouveau_score(row):
    global scores  # Utiliser le dictionnaire global des scores
    p1 = row['Player1Id']
    p2 = row['Player2Id']
    winner = row['WinnerId']
    
    # Ancien scores
    score_p1 = scores.get(p1, 1000)
    score_p2 = scores.get(p2, 1000)
    
    # Points calculés
    base_points = row['Base Points']
    points_tour = row['Encoded Round']
    points_temps = 1 + row['Encoded Duration']
    prg_p1 = row['PRG P1']
    prg_p2 = row['PRG P2']
    
    # Coefficients adversaire
    coef_p1 = coef_adversaire(row['Classement P1'], row['Classement P2'])
    coef_p2 = coef_adversaire(row['Classement P2'], row['Classement P1'])
    
    # Calcul du nouveau score
    if winner == p1:
        nouveau_score_p1 = score_p1 + (base_points + points_tour) * prg_p1 * points_temps * coef_p1
        nouveau_score_p2 = score_p2 - (base_points + points_tour) * prg_p2 * points_temps * coef_p2
    elif winner == p2:
        nouveau_score_p2 = score_p2 + (base_points + points_tour) * prg_p2 * points_temps * coef_p2
        nouveau_score_p1 = score_p1 - (base_points + points_tour) * prg_p1 * points_temps * coef_p1
    else:
        nouveau_score_p1, nouveau_score_p2 = score_p1, score_p2  # Aucun changement
    
    # Mise à jour des scores globaux
    scores[p1] = max(0, nouveau_score_p1)
    scores[p2] = max(0, nouveau_score_p2)
    
    return nouveau_score_p1, nouveau_score_p2

# Appliquer la fonction pour chaque match
df[['Nouveau Score P1', 'Nouveau Score P2']] = df.apply(calculer_nouveau_score, axis=1, result_type='expand')

# Affichage des résultats
print(df[['Player1Id', 'Player2Id', 'WinnerId', 'Nouveau Score P1', 'Nouveau Score P2']].head(10))


   Player1Id  Player2Id  WinnerId  Nouveau Score P1  Nouveau Score P2
0      12799      12787     12787        963.675676       1047.675676
1      12769      12776     12776        950.720000       1062.720000
2      12821      12787     12821       1056.486957        992.162632
3      12821      12784     12821       1104.815724        964.328767
4      12798      12792     12792        965.690141       1049.690141
5      12766      12811     12766       1045.073171        961.073171
6      12766      12785     12766       1079.160127        978.086957
7      12767      12785     12785        946.352941       1036.439898
8      12764      12782     12764       1048.328767        964.328767
9      12802      12764     12802       1064.842105       1001.170872


In [12]:
df['Victory P1'] = df['WinnerId'] == df['Player1Id']
y = df['Victory P1']  # Cible de la régression logistique
X = df[[ 'Encoded Round', 'Encoded Duration',
        'Victoires Cumulées P1', 'Victoires Cumulées P2', 
        ]]


In [13]:
# Vérifier la présence de NaN dans les données
print(X.isna().sum())  # Pour X
print(y.isna().sum())  # Pour y





Encoded Round            0
Encoded Duration         0
Victoires Cumulées P1    0
Victoires Cumulées P2    0
dtype: int64
0


In [14]:
X = X.fillna(0)
print(X.isna().sum())  # Pour X
print(y.isna().sum()) 

Encoded Round            0
Encoded Duration         0
Victoires Cumulées P1    0
Victoires Cumulées P2    0
dtype: int64
0


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialiser le modèle de régression logistique avec max_iter=1000 et le solveur 'liblinear'
model = LogisticRegression(max_iter=1000, solver='liblinear')

# Entraîner le modèle
model.fit(X_train_scaled, y_train)

# Faire des prédictions
y_pred = model.predict(X_test_scaled)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
# Afficher les coefficients du modèle pour évaluer l'importance des caractéristiques
print("Coefficients du modèle : ", model.coef_)




Accuracy: 72.39%
Coefficients du modèle :  [[ 0.03581334 -0.01890036  1.13039586 -1.05911137]]


In [17]:
X = df[[ 'Encoded Round', 'Encoded Duration',
        'Victoires Cumulées P1', 'Victoires Cumulées P2', 
        'PRG P1', 'PRG P2']]

X = X.fillna(0)
print(X.isna().sum())


Encoded Round            0
Encoded Duration         0
Victoires Cumulées P1    0
Victoires Cumulées P2    0
PRG P1                   0
PRG P2                   0
dtype: int64


In [18]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardiser les données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialiser le modèle de régression logistique avec max_iter=1000 et le solveur 'liblinear'
model = LogisticRegression(max_iter=1000, solver='liblinear')

# Entraîner le modèle
model.fit(X_train_scaled, y_train)

# Faire des prédictions
y_pred = model.predict(X_test_scaled)

# Calculer l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
# Afficher les coefficients du modèle pour évaluer l'importance des caractéristiques
print("Coefficients du modèle : ", model.coef_)


Accuracy: 95.40%
Coefficients du modèle :  [[ 0.00794554 -0.02654474  0.16776826 -0.13150767  6.702039   -6.66760379]]
